In [2]:
!pip install rectools > None

In [3]:
import dill
import numpy as np
import yaml
import cProfile
import timeit

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd gdrive/MyDrive/recsys/

/content/gdrive/MyDrive/recsys


In [6]:
class LightFMWrapper:
    def __init__(self):
        self.model = None
        self.users_mapping, self.items_inv_mapping = None, None
        self.__load_models()

    def recommend(self, user_id, n_recs=10):
        user_inner_idx = self.users_mapping[user_id[0]]
        items_embedding, user_embedding = self.__get_embeddings(user_inner_idx)
        scores = items_embedding @ user_embedding
        top_score_ids = scores.argsort()[-n_recs:][::-1]
        items_to_recommend = [
            self.items_inv_mapping[item] for item in top_score_ids if
            item in self.items_inv_mapping
        ]
        return items_to_recommend

    def __get_embeddings(self, user_inner_idx):
        user_biases, user_embedding = (
            self.model.get_user_representations()[0][user_inner_idx],
            self.model.get_user_representations()[1][user_inner_idx],
        )
        items_biases, items_embedding = self.model.get_item_representations()
        items_embedding = items_embedding[:len(self.items_inv_mapping), :]
        items_biases = items_biases[:len(self.items_inv_mapping)]
        user_embedding = np.hstack(
            (
                user_biases, np.ones(user_biases.size),
                user_embedding,
            ),
        )
        items_embedding = np.hstack(
            (
                np.ones((items_biases.size, 1)),
                items_biases[:, np.newaxis],
                items_embedding,
            ),
        )
        return items_embedding, user_embedding

    def __load_models(self):
        try:
            with open('lightfm.dill', 'rb') as f:
                self.model = dill.load(f)

            with open(
                'users_mapping.dill',
                'rb',
            ) as f:
                self.users_mapping = dill.load(f)

            with open(
                'items_inv_mapping.dill',
                'rb',
            ) as f:
                self.items_inv_mapping = dill.load(f)
        except FileNotFoundError:
            print('models folder is empty...')



In [7]:
lightfm_model = LightFMWrapper()

In [19]:
testcode = ''' 
for user_id in list(lightfm_model.users_mapping.keys())[:400]: 
  lightfm_model.recommend([user_id])
'''

t = timeit.timeit(testcode, "from __main__ import lightfm_model", number=10)
print(f'Average time: {t / 10:0.5f} s')

Average time: 1.45671 s
